In [1]:
from transformers import BertTokenizer
  
tokenizer = BertTokenizer.from_pretrained("cl-tohoku/bert-base-japanese", do_lower_case=True)
text = '東京都は2日、都内で新たに34人が新型コロナウイルスに感染していることを確認したと発表しました。1日の感染の確認が30人以上となるのは先月14日以来、19日ぶりです。'
tokenized_text = tokenizer.tokenize(text)
print(tokenized_text)


['東', '京', '都', 'は', '##2', '日', '、', '都', '内', 'て', '新', 'た', '##に', '##3', '##4', '人', 'か', '新', '型', 'コロナ', '##ウイルス', '##に', '感', '染', 'し', '##てい', '##る', '##こと', '##を', '確', '認', 'した', '##と', '発', '表', 'しま', '##した', '。', '1', '日', 'の', '感', '染', 'の', '確', '認', 'か', '##30', '人', '以', '上', 'と', '##なる', '##の', '##は', '先', '月', '14', '日', '以', '来', '、', '19', '日', 'ふり', '##て', '##す', '。']


In [3]:
# import tqdm
# from transformers import BertTokenizer

tokenizer_char_whole_word_masking = BertTokenizer.from_pretrained("cl-tohoku/bert-base-japanese-char-whole-word-masking", do_lower_case=True)
text = '東京都は2日、都内で新たに34人が新型コロナウイルスに感染していることを確認したと発表しました。1日の感染の確認が30人以上となるのは先月14日以来、19日ぶりです。'
tokenized_text = tokenizer_char_whole_word_masking.tokenize(text)
print(tokenized_text)


['東', '京', '都', '[UNK]', '日', '、', '都', '内', 'て', '新', '[UNK]', '人', 'か', '新', '型', '[UNK]', '感', '染', '[UNK]', '確', '認', '[UNK]', '発', '表', '[UNK]', '。', '1', '日', 'の', '感', '染', 'の', '確', '認', '[UNK]', '人', '以', '上', '[UNK]', '先', '月', '[UNK]', '日', '以', '来', '、', '[UNK]', '日', '[UNK]', '。']


In [1]:
import os

data_folder = './release/train/ca_data'

arr = os.listdir(data_folder)
print(arr)


['300338457.pdf.xlsx', '300351401.pdf.xlsx', '300340065.pdf.xlsx', '300351386.pdf.xlsx', '300351441.pdf.xlsx', '300340176.pdf.xlsx', '300351404.pdf.xlsx', '300351413.pdf.xlsx', '300351416.pdf.xlsx', '300351396.pdf.xlsx', '300359888.pdf.xlsx', '300359891.pdf.xlsx', '300359894.pdf.xlsx', '300359897.pdf.xlsx', '300359904.pdf.xlsx', '300359906.pdf.xlsx', '300359909.pdf.xlsx', '300359954.pdf.xlsx', '300359382.pdf.xlsx', '300359722.pdf.xlsx', '300359381.pdf.xlsx', '300360246.pdf.xlsx', '300360248.pdf.xlsx', '300364785.pdf.xlsx', '300364838.pdf.xlsx', '300365038.pdf.xlsx', '300365063.pdf.xlsx', '300365182.pdf.xlsx', '300365613.pdf.xlsx', '300365614.pdf.xlsx', '300365616.pdf.xlsx', '300365617.pdf.xlsx', '300365618.pdf.xlsx', '300365621.pdf.xlsx', '300365622.pdf.xlsx', '300365624.pdf.xlsx', '300365084.pdf.xlsx', '300365625.pdf.xlsx', '300365626.pdf.xlsx', '300365664.pdf.xlsx', '300365669.pdf.xlsx', '300365697.pdf.xlsx', '300365753.pdf.xlsx', '300365795.pdf.xlsx', '300365804.pdf.xlsx', '30036585

In [162]:
import pandas as pd
from tqdm import tqdm

# pd.read_excel(os.path.join(data_folder, arr[0]), header=0)

df_train = []
df_list = []

for filepath in tqdm(arr):
    df = pd.read_excel(os.path.join(data_folder, filepath), header=0)
    df_list.append(df)
    
df_train = pd.concat(df_list, axis=0, ignore_index=True)  
del df_list

# df_train


100%|██████████| 82/82 [00:00<00:00, 118.45it/s]


In [163]:
import unicodedata
import re

tag_map = {
    "調達年度": 1,
    "都道府県": 2,
    "入札件名": 3,
    "施設名": 4,
    "需要場所(住所)": 5,
    "調達開始日": 6,
    "調達終了日": 7,	
    "公告日": 8,
    "仕様書交付期限": 9,	
    "質問票締切日時": 10,	
    "資格申請締切日時": 11,	
    "入札書締切日時": 12,
    "開札日時": 13,	
    "質問箇所所属/担当者": 14,
    "質問箇所TEL/FAX": 15,	
    "資格申請送付先": 16,
    "資格申請送付先部署/担当者名": 17,	
    "入札書送付先": 18,	
    "入札書送付先部署/担当者名": 19,
    "開札場所": 20
}

def normalize_data(df_train):
    df_train['Tag'] = df_train['Tag'].map(lambda x: unicodedata.normalize("NFKC", re.sub('＊|\*|\s+', '', x)).split(';') if pd.notnull(x) else [])
    df_train['Tag_n'] = df_train['Tag'].map(lambda x: [1 if (type(x) != float and i in x) else 0 for i in tag_map])
    # 1 if (type(x) != float and i in x) else 0 
#     df_train['Tag_n'] = df_train['Tag'].map(lambda x: [1 if (i in x) else 0 for i in tag_map])

    df_train['Text'] = df_train['Text'].map(lambda x: unicodedata.normalize("NFKC", re.sub('＊|\*|\s+', '', x)))
    df_train['Value'] = df_train['Value'].map(lambda x: unicodedata.normalize("NFKC", re.sub('＊|\*|\s+', '', x)) if pd.notnull(x) else '')
    
    df_train['y'] = df_train.apply(lambda x: [0]*50, axis=1)
    
    return df_train

In [164]:
df_train_norm = normalize_data(df_train)

# df_train_norm
# df_train_norm
# df_train[["Value", "Tag_n"]]
# df_train[["Value"]]
# print([i for i in tag_map])
# print('調達年度' in ['調達年度' ,'入札件名', '施設名', '需要場所(住所)'])


In [168]:
df_train_norm[0:15]


,Page No,Text,Index,Parent Index,Is Title,Is Table,Tag,Value,Tag_n,y
0,1.0,入札公告,1,NaN,x,NaN,[],,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,1.0,次のとおり一般競争入札に付します。,2,1.0,NaN,NaN,[],,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,1.0,平成29年9月22日,3,1.0,NaN,NaN,[公告日],平成29年9月22日,"[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,1.0,独立行政法人石油天然ガス・金属鉱物資源機構,4,1.0,NaN,NaN,[],,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,1.0,契約担当役,5,1.0,NaN,NaN,[],,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
5,1.0,金属・石炭事業支援本部長池田肇,6,1.0,NaN,NaN,[],,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
6,1.0,◎調達機関番号586◎所在地番号13,7,1.0,NaN,NaN,[],,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
7,1.0,1調達内容,8,1.0,x,NaN,[],,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
8,1.0,(1)品目分類番号26,9,8.0,NaN,NaN,[],,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
9,1.0,(2)購入等件名及び予定数量旧松尾鉱山新中和処理施設で使用する電気契約電力570kW,10,8.0,NaN,NaN,[入札件名],旧松尾鉱山新中和処理施設で使用する電気,"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [41]:
print(df_train[3:400])

     Page No                                 Text  Index  Parent Index  \
3        1.0                独立行政法人石油天然ガス・金属鉱物資源機構      4           1.0   
4        1.0                                契約担当役      5           1.0   
5        1.0                    金属・石炭事業支援本部長 池田 肇      6           1.0   
6        1.0                ◎調達機関番号 ５８６ ◎所在地番号 １３      7           1.0   
7        1.0                               １ 調達内容      8           1.0   
..       ...                                  ...    ...           ...   
395      3.0               り競争入札参加申し込みを行った者であること。     67          66.0   
396      3.0  （８）入札説明書の配布資料３に定めた「供給能力審査書類」（様式Ａ）及び     68          46.0   
397      3.0    「適合証明書」（様式Ｂ）を提出し、かつ内容が妥当であると認められた     69          68.0   
398      3.0                              者であること。     70          68.0   
399      3.0                         ３．競争入札参加申し込み     71           1.0   

    Is Title Is Table  Tag Value  
3        NaN      NaN  NaN   NaN  
4        NaN      NaN  NaN   NaN  
5     

In [99]:
from transformers import BertTokenizer
  
tokenizer = BertTokenizer.from_pretrained("cl-tohoku/bert-base-japanese", do_lower_case=True)

for idx, df in df_train[0:50].iterrows():
#     print(idx)
#     print(df)
    print("text:")
    print(df["Text"])
    print(tokenizer.tokenize(df["Text"]))
    if type(df["Value"]) != float: 
        print("Tag")
        print(df["Tag"])
        print("value:")
        print(df["Value"])
        value_list = df["Value"].split(";")
        print([tokenizer.tokenize(sub_val) for sub_val in value_list])
    print("NEXT\n")

text:
入札公告
['入', '札', '公', '告']
NEXT

text:
次のとおり一般競争入札に付します。
['次', 'の', '##と', '##おり', '一', '般', '競', '争', '入', '札', 'に', '付', 'しま', '##す', '。']
NEXT

text:
平成29年9月22日
['平', '成', '29', '年', '9', '月', '22', '日']
Tag
公告日
value:
平成29年9月22日
[['平', '成', '29', '年', '9', '月', '22', '日']]
NEXT

text:
独立行政法人石油天然ガス・金属鉱物資源機構
['独', '立', '行', '政', '法', '人', '石', '油', '天', '然', 'カス', '・', '金', '属', '鉱', '物', '資', '源', '機', '構']
NEXT

text:
契約担当役
['契', '約', '担', '当', '役']
NEXT

text:
金属・石炭事業支援本部長 池田 肇
['金', '属', '・', '石', '炭', '事', '業', '支', '援', '本', '部', '長', '池', '田', '肇']
NEXT

text:
◎調達機関番号 ５８６ ◎所在地番号 １３
['[UNK]', '調', '達', '機', '関', '番', '号', '[UNK]', '[UNK]', '所', '在', '地', '番', '号', '[UNK]']
NEXT

text:
１ 調達内容
['[UNK]', '調', '達', '内', '容']
NEXT

text:
（１）品目分類番号 ２６
['[UNK]', '[UNK]', '[UNK]', '品', '目', '分', '類', '番', '号', '[UNK]']
NEXT

text:
（２）購入等件名及び予定数量 旧松尾鉱山新中和処 理 施 設で使用する電気 契 約 電 力 570kW
['[UNK]', '[UNK]', '[UNK]', '購', '入', '等', '件', '名', '及', 'ひ', '予', '定', '数', '量', '旧', '松', '尾', '鉱

In [85]:
pd.notnull(0)

TypeError: notnull() takes 1 positional argument but 2 were given